In [1]:
import torch
from GASU_env import GASU
from GASU_optimizer import optimize_GASU

In [2]:
# Episode Length: T
T = 31
state_horizon = 30
action_horizon = 1
simulation_days = T + state_horizon - action_horizon       # Total simulation days = 60

env_id = 'GASU-v0'
# env = GASU(env_id)

env = GASU(env_id, config_path="gasu_config.json")
# Rendering essential information from environment
compressors = env.compressor_info()       # Provides the infromation compressors and its initial state in dict format
external_purchase_price = env.get_external_purchase_price()    # Provides the dict of external purchase price

# initializing the optimizer by feeding the essential information from environment
optimizer = optimize_GASU(state_horizon, action_horizon, compressors)
optimizer._update_external_purchase_price(external_purchase_price)         # __Update the external purchase price in optimizer__

### Simulation for 31 days (with optimal action)

In [19]:
env.reset()
flatt_state = env._get_state('flatt')  # mode = 'flatt'
opt_episode_reward = 0
simulation_episode_reward = 0
# optimal_episode_cost = 0
max_purchase =0

terminated = torch.tensor(False)
while terminated.item() is False:

    optimizer.update_state(flatt_state)
    objective_value = optimizer.solve()
    opt_cost = optimizer.action_horizon_cost()
    opt_action = optimizer._fetch_optimal_action_for_action_horizon()
    max_ext_purchase = opt_action[-1]
    if int(max_ext_purchase) > max_purchase:
        max_purchase = int(max_ext_purchase)
    # feed action to environment, # tensor output
    flatt_state_tensor, reward, terminated, truncated, info = env.step(opt_action)
    flatt_state = env._get_state('flatt')
    opt_episode_reward += opt_cost
    simulation_episode_reward += reward.item()
    
print(f"Optimal Episode Reward: {opt_episode_reward}")
print(f"Final Episode Reward: {simulation_episode_reward}")
print(f"Max External Purchase: {max_purchase}")

Optimal Episode Reward: 1232.8234047772164
Final Episode Reward: -1232.8234034398854
Max External Purchase: 519


In [ ]:
# update max_purchase by checking the maximum value in the max_ext_purchase array
if max_ext_purchase.max() > max_purchase:
    max_purchase = max_ext_purchase.max()

In [8]:
opt_action

array([0.        , 0.        , 0.        , 0.77541333, 0.        ,
       1.        , 0.        ], dtype=float32)

In [15]:
type(opt_action[-1])


numpy.float32

In [17]:
int(opt_action[-1])

0